In [1]:
using DataStructures
using CPUTime
using DelimitedFiles
using Base
using Distributed

In [9]:
numVertex=4
Edges = [(4,1,2), (1,2,1), (2,3,1), (3,4,1), (4,2,3), (1,3,5)]

6-element Array{Tuple{Int64,Int64,Int64},1}:
 (4, 1, 2)
 (1, 2, 1)
 (2, 3, 1)
 (3, 4, 1)
 (4, 2, 3)
 (1, 3, 5)

In [3]:
numVertex=6
Edges = [
(6, 1, 1),
(6, 2, 3),
(1, 2, 1),
(1, 3, 7),
(2, 4, 1),
(1, 4, 3),
(3, 4, 1),
(3, 5, 1),
(4, 5, 6)]
# source, destionation, weight

9-element Array{Tuple{Int64,Int64,Int64},1}:
 (6, 1, 1)
 (6, 2, 3)
 (1, 2, 1)
 (1, 3, 7)
 (2, 4, 1)
 (1, 4, 3)
 (3, 4, 1)
 (3, 5, 1)
 (4, 5, 6)

In [2]:
Edges = Vector{}()
function readDataFromFile(filename="in7.txt")
    open(filename, "r") do f 
        n, m = split(readline(f))
        n = parse(Int, n)
        sizehint!(Edges, parse(Int, m))
#         println(n," ",m)
        for ln in eachline(f)
#             a,b,c = split(ln)
            a,b,c = readdlm(IOBuffer(ln), Int)
            if(a==0)
                a=n
            end
            if(b==0)
                b=n
            end
            push!(Edges, (a,b,c))
#             println(a," ",b," ",c)
        end
        return n
    end
end
numVertex = readDataFromFile()

10000

In [3]:
uf = IntDisjointSets(numVertex+1)

weights = Vector{}()
sizehint!(weights, length(Edges))

mst= Vector{}()
sizehint!(mst, numVertex)

0-element Array{Any,1}

In [4]:
for e in Edges
    push!(weights, (e[3], (src=e[1],dst= e[2])))
end

In [5]:
Base.@pure function kruskal(EdgeWeights, MST, UF)
    if isempty(EdgeWeights)
        return 0
    end
    sortedEdgeWeights = sort(EdgeWeights)
    mst_weight=0
    for weight in sortedEdgeWeights
        c, e = weight
        if !in_same_set(uf, e.src, e.dst)
            union!(uf, e.src, e.dst)
            push!(mst, e)    
            mst_weight+=c
        end
    end
    return mst_weight
end    

kruskal (generic function with 1 method)

In [6]:
using Random
rng = MersenneTwister();

In [7]:
Base.@pure function filterKruskal(EdgeWeights, MST, UF)
    if isempty(EdgeWeights)
        return 0
    end
    if kruskalThreshold(length(EdgeWeights), length(MST))
        # println("pre-start kruskal ",length(EdgeWeights) ," at", CPUtime_us())
        tmp =  @async kruskal(EdgeWeights, MST, UF)
        # println("allocated kruskal ",length(EdgeWeights) ," at", CPUtime_us())
        res = fetch(tmp)
        # println("finished kruskal ",length(EdgeWeights) ," at", CPUtime_us())
        return res
    else
        sampler = Random.Sampler(rng, EdgeWeights)
        pivot = rand(rng, sampler)
        
        # Partition
        EdgeWeightsLeft = Vector{}()
        sizehint!(EdgeWeightsLeft, length(EdgeWeights)÷2)

        EdgeWeightsRight = Vector{}()
        sizehint!(EdgeWeightsRight, length(EdgeWeights)÷2)

        for EdgeWeight in EdgeWeights
            if EdgeWeight[1] < pivot[1]
                push!(EdgeWeightsLeft, EdgeWeight)
            else
                push!(EdgeWeightsRight, EdgeWeight)
            end
        end
        
#         pmap(x->(x[1]<pivot[1]) ? push!(EdgeWeightsLeft, x) : push!(EdgeWeightsRight, x), EdgeWeights)
#         println("pivot = ",pivot)
#         println("left = ",length(EdgeWeightsLeft))
#         println("right = ",length(EdgeWeightsRight))
#         println("Partition ",length(EdgeWeightsLeft)," ",length(EdgeWeightsRight) ," going to start at ", CPUtime_us())
        leftMST = @async filterKruskal(EdgeWeightsLeft, MST, UF)
        FilteredEdgeWeightsRight = filter(EdgeWeightsRight, UF)
        rightMST = @async filterKruskal(FilteredEdgeWeightsRight, MST, UF)
        # println("Partition ",length(EdgeWeightsLeft)," ",length(EdgeWeightsRight) ," allocated at ", CPUtime_us())
        res = fetch(leftMST) + fetch(rightMST)
        # println("Partition ",length(EdgeWeightsLeft)," ",length(EdgeWeightsRight) ," finished at ", CPUtime_us())
        return res
    end
end

filterKruskal (generic function with 1 method)

In [8]:
Base.@pure function filterKruskal(EdgeWeights, MST, UF)
    if isempty(EdgeWeights)
        return 0
    end
    if kruskalThreshold(length(EdgeWeights), length(MST))
        # println("pre-start kruskal ",length(EdgeWeights) ," at", CPUtime_us())
        tmp = kruskal(EdgeWeights, MST, UF)
        # println("allocated kruskal ",length(EdgeWeights) ," at", CPUtime_us())
        res = tmp
        # println("finished kruskal ",length(EdgeWeights) ," at", CPUtime_us())
        return res
    else
        sampler = Random.Sampler(rng, EdgeWeights)
        pivot = rand(rng, sampler)
        
        # Partition
        EdgeWeightsLeft = Vector{}()
        sizehint!(EdgeWeightsLeft, length(EdgeWeights)÷2)

        EdgeWeightsRight = Vector{}()
        sizehint!(EdgeWeightsRight, length(EdgeWeights)÷2)

        for EdgeWeight in EdgeWeights
            if EdgeWeight[1] < pivot[1]
                push!(EdgeWeightsLeft, EdgeWeight)
            else
                push!(EdgeWeightsRight, EdgeWeight)
            end
        end
        
#         pmap(x->(x[1]<pivot[1]) ? push!(EdgeWeightsLeft, x) : push!(EdgeWeightsRight, x), EdgeWeights)
#         println("pivot = ",pivot)
#         println("left = ",length(EdgeWeightsLeft))
#         println("right = ",length(EdgeWeightsRight))
#         println("Partition ",length(EdgeWeightsLeft)," ",length(EdgeWeightsRight) ," going to start at ", CPUtime_us())
        leftMST = filterKruskal(EdgeWeightsLeft, MST, UF)
        FilteredEdgeWeightsRight = filter(EdgeWeightsRight, UF)
        rightMST = filterKruskal(FilteredEdgeWeightsRight, MST, UF)
        # println("Partition ",length(EdgeWeightsLeft)," ",length(EdgeWeightsRight) ," allocated at ", CPUtime_us())
        res = (leftMST) + (rightMST)
        # println("Partition ",length(EdgeWeightsLeft)," ",length(EdgeWeightsRight) ," finished at ", CPUtime_us())
        return res
    end
end

filterKruskal (generic function with 1 method)

In [9]:
Base.@pure function kruskalThreshold(EdgesLength, MSTLength)
    sp = Random.Sampler(rng, 1:EdgesLength) # or Random.Sampler(MersenneTwister, 1:20)
#     return EdgesLength <=10000
    return EdgesLength/1.25<=rand(rng, sp) || EdgesLength <=1000
end

kruskalThreshold (generic function with 1 method)

In [10]:
Base.@pure function filter(EdgeWeights, UF)
    FilteredEdgeWeights = Vector{}()
    for EdgeWeight in EdgeWeights
        if !in_same_set(UF, EdgeWeight[2].src, EdgeWeight[2].dst)
            push!(FilteredEdgeWeights, EdgeWeight)
        end
    end
    return FilteredEdgeWeights
end

filter (generic function with 1 method)

In [51]:
@time @CPUtime filterKruskal(weights, mst, uf)

elapsed CPU time: 0.546 seconds
  0.544201 seconds (1.93 M allocations: 49.558 MiB, 9.50% gc time)


6064035

In [11]:
@time @CPUtime kruskal(weights, mst, uf)


elapsed CPU time: 0.240698 seconds
  0.324077 seconds (1.77 M allocations: 52.194 MiB, 7.67% gc time)


6064035

In [12]:
Threads.nthreads()

4

In [8]:
println(length(mst))

99999


In [13]:
println(mst)

Any[(src = 263, dst = 5145), (src = 314, dst = 9268), (src = 325, dst = 7260), (src = 1294, dst = 7502), (src = 1624, dst = 6662), (src = 2945, dst = 9736), (src = 5917, dst = 9213), (src = 6162, dst = 9338), (src = 7188, dst = 4722), (src = 7302, dst = 6191), (src = 7495, dst = 7860), (src = 7646, dst = 8688), (src = 8132, dst = 2199), (src = 8260, dst = 1487), (src = 881, dst = 8174), (src = 7923, dst = 6652), (src = 9813, dst = 8126), (src = 1038, dst = 6166), (src = 1630, dst = 2880), (src = 1809, dst = 6726), (src = 1993, dst = 7032), (src = 2232, dst = 7650), (src = 3443, dst = 4623), (src = 3672, dst = 2024), (src = 5751, dst = 1076), (src = 7066, dst = 6879), (src = 9432, dst = 3297), (src = 704, dst = 2477), (src = 2307, dst = 2678), (src = 2612, dst = 1264), (src = 8643, dst = 4956), (src = 4745, dst = 9223), (src = 5393, dst = 9149), (src = 5982, dst = 5645), (src = 9715, dst = 832), (src = 2700, dst = 1794), (src = 4544, dst = 3962), (src = 4996, dst = 9738), (src = 8795, d

, dst = 2693), (src = 1218, dst = 2042), (src = 2315, dst = 5038), (src = 2531, dst = 4962), (src = 5770, dst = 129), (src = 5908, dst = 8941), (src = 7722, dst = 3295), (src = 7831, dst = 5015), (src = 8925, dst = 8067), (src = 9519, dst = 7946), (src = 9885, dst = 3615), (src = 2717, dst = 1139), (src = 3180, dst = 7198), (src = 5900, dst = 5135), (src = 6689, dst = 3626), (src = 8323, dst = 4380), (src = 513, dst = 7680), (src = 710, dst = 8907), (src = 900, dst = 7639), (src = 2789, dst = 278), (src = 6712, dst = 383), (src = 8193, dst = 8084), (src = 8708, dst = 441), (src = 9399, dst = 7015), (src = 9524, dst = 1719), (src = 9558, dst = 8235), (src = 141, dst = 2057), (src = 3808, dst = 1696), (src = 6880, dst = 8517), (src = 7100, dst = 1004), (src = 7384, dst = 1242), (src = 7735, dst = 4378), (src = 7884, dst = 2341), (src = 9520, dst = 5603), (src = 2340, dst = 4261), (src = 2614, dst = 4180), (src = 2825, dst = 222), (src = 3732, dst = 2444), (src = 5136, dst = 6173), (src =

(src = 3263, dst = 1069), (src = 4714, dst = 2104), (src = 6308, dst = 1689), (src = 6473, dst = 8342), (src = 6514, dst = 5538), (src = 6901, dst = 1131), (src = 8001, dst = 4903), (src = 8657, dst = 2402), (src = 8672, dst = 702), (src = 8797, dst = 1328), (src = 1832, dst = 3606), (src = 1871, dst = 3923), (src = 2173, dst = 2973), (src = 2878, dst = 5848), (src = 2890, dst = 7022), (src = 3519, dst = 5323), (src = 3623, dst = 8290), (src = 3830, dst = 8277), (src = 4398, dst = 7816), (src = 5490, dst = 7381), (src = 6001, dst = 1492), (src = 6111, dst = 1963), (src = 535, dst = 5280), (src = 1542, dst = 3859), (src = 2276, dst = 638), (src = 2405, dst = 2371), (src = 4744, dst = 9597), (src = 5774, dst = 2495), (src = 6269, dst = 6858), (src = 6657, dst = 7232), (src = 7022, dst = 1462), (src = 7689, dst = 8684), (src = 8426, dst = 6334), (src = 9745, dst = 7159), (src = 921, dst = 8349), (src = 993, dst = 5064), (src = 1796, dst = 7423), (src = 2412, dst = 1719), (src = 2653, dst 

 = 9034, dst = 5647), (src = 9163, dst = 4739), (src = 9838, dst = 1581), (src = 3266, dst = 4715), (src = 4379, dst = 9746), (src = 4707, dst = 2706), (src = 4805, dst = 6567), (src = 5247, dst = 1070), (src = 9068, dst = 2310), (src = 9258, dst = 4338), (src = 348, dst = 5106), (src = 653, dst = 2679), (src = 1925, dst = 9896), (src = 6634, dst = 5574), (src = 6655, dst = 4110), (src = 8144, dst = 4547), (src = 8290, dst = 6739), (src = 64, dst = 10), (src = 351, dst = 9983), (src = 1041, dst = 8788), (src = 1337, dst = 6966), (src = 4614, dst = 926), (src = 8075, dst = 2407), (src = 8147, dst = 1443), (src = 873, dst = 5296), (src = 1523, dst = 1753), (src = 1530, dst = 9976), (src = 1934, dst = 6369), (src = 2264, dst = 4983), (src = 3809, dst = 9351), (src = 6255, dst = 7111), (src = 6589, dst = 682), (src = 7018, dst = 2690), (src = 7318, dst = 4717), (src = 7724, dst = 8680), (src = 8205, dst = 5626), (src = 8604, dst = 4579), (src = 9299, dst = 9708), (src = 9682, dst = 8276), 

8364, dst = 2279), (src = 1298, dst = 1113), (src = 1606, dst = 4058), (src = 2534, dst = 370), (src = 3825, dst = 9892), (src = 5128, dst = 1430), (src = 5774, dst = 7989), (src = 6801, dst = 929), (src = 6888, dst = 9303), (src = 6983, dst = 5155), (src = 8440, dst = 482), (src = 9498, dst = 1590), (src = 1036, dst = 1152), (src = 1073, dst = 3015), (src = 1240, dst = 5878), (src = 2100, dst = 2540), (src = 2514, dst = 5972), (src = 2519, dst = 5374), (src = 2665, dst = 6643), (src = 2926, dst = 7056), (src = 3860, dst = 1799), (src = 4403, dst = 5826), (src = 5454, dst = 9748), (src = 8729, dst = 4930), (src = 9621, dst = 7475), (src = 487, dst = 5050), (src = 897, dst = 5477), (src = 1512, dst = 2352), (src = 1593, dst = 5537), (src = 1625, dst = 2263), (src = 1867, dst = 915), (src = 2341, dst = 6202), (src = 5733, dst = 802), (src = 9346, dst = 464), (src = 70, dst = 9062), (src = 2751, dst = 4743), (src = 3636, dst = 1817), (src = 5263, dst = 7643), (src = 6854, dst = 2043), (sr

...

), (src = 5580, dst = 5094), (src = 1596, dst = 7540), (src = 1616, dst = 1244), (src = 5734, dst = 4309), (src = 7269, dst = 8285), (src = 8739, dst = 6409), (src = 3912, dst = 7180), (src = 1304, dst = 2972), (src = 4173, dst = 8176), (src = 4272, dst = 1490), (src = 5000, dst = 218), (src = 5653, dst = 6255), (src = 9893, dst = 701), (src = 560, dst = 7909), (src = 1853, dst = 5401), (src = 4005, dst = 8707), (src = 6003, dst = 2490), (src = 8386, dst = 6507), (src = 4165, dst = 382), (src = 4712, dst = 6365), (src = 4718, dst = 3987), (src = 6375, dst = 1429), (src = 5721, dst = 1915), (src = 8115, dst = 9599), (src = 587, dst = 1966), (src = 3876, dst = 6992), (src = 5575, dst = 6939), (src = 6155, dst = 5865), (src = 9351, dst = 9036), (src = 21, dst = 357), (src = 539, dst = 8211), (src = 17, dst = 8481), (src = 5087, dst = 2545), (src = 3622, dst = 2369), (src = 5319, dst = 6800), (src = 2701, dst = 3591), (src = 322, dst = 1316), (src = 4363, dst = 4985), (src = 9277, dst = 34

dst = 4462), (src = 3618, dst = 457), (src = 8580, dst = 7971), (src = 35, dst = 5770), (src = 4425, dst = 506), (src = 5479, dst = 8995), (src = 8637, dst = 4284), (src = 9613, dst = 7346), (src = 9643, dst = 5064), (src = 3836, dst = 424), (src = 7308, dst = 1206), (src = 1153, dst = 3522), (src = 3174, dst = 9141), (src = 5741, dst = 6563), (src = 6897, dst = 5294), (src = 2318, dst = 5698), (src = 9146, dst = 3685), (src = 9978, dst = 1268), (src = 1622, dst = 4564), (src = 8743, dst = 1262), (src = 9324, dst = 9449), (src = 9472, dst = 9008), (src = 565, dst = 6443), (src = 2441, dst = 8928), (src = 8857, dst = 691), (src = 9664, dst = 1895), (src = 9839, dst = 4797), (src = 5476, dst = 8971), (src = 6931, dst = 8713), (src = 8125, dst = 2640), (src = 8126, dst = 7611), (src = 661, dst = 4281), (src = 5737, dst = 9540), (src = 7811, dst = 9070), (src = 8468, dst = 8836), (src = 7609, dst = 5730), (src = 8843, dst = 5559), (src = 8983, dst = 6233), (src = 5668, dst = 2427), (src = 

 = 259), (src = 7858, dst = 8618), (src = 7693, dst = 5670), (src = 8353, dst = 5619), (src = 8622, dst = 4047), (src = 2279, dst = 9617), (src = 4893, dst = 2917), (src = 6654, dst = 8515), (src = 5337, dst = 1572), (src = 6682, dst = 9306), (src = 4099, dst = 4383), (src = 5811, dst = 4057), (src = 8127, dst = 2894), (src = 8579, dst = 9928), (src = 2263, dst = 334), (src = 1485, dst = 9845), (src = 4901, dst = 9545), (src = 6013, dst = 3679), (src = 221, dst = 1936), (src = 769, dst = 964), (src = 8923, dst = 8313), (src = 2349, dst = 3445), (src = 7310, dst = 6867), (src = 8102, dst = 5495), (src = 3896, dst = 5263), (src = 173, dst = 8785), (src = 2498, dst = 2230), (src = 2884, dst = 7720), (src = 4154, dst = 2641), (src = 9192, dst = 6393), (src = 7791, dst = 9559), (src = 433, dst = 7404), (src = 3289, dst = 9910), (src = 4062, dst = 5021), (src = 3934, dst = 3541), (src = 7532, dst = 1025), (src = 8976, dst = 2967), (src = 9504, dst = 5118), (src = 3223, dst = 5585), (src = 52

 = 9205, dst = 4033), (src = 4785, dst = 2697), (src = 8460, dst = 7173), (src = 4187, dst = 1698), (src = 7047, dst = 8192), (src = 99, dst = 6524), (src = 3523, dst = 6018), (src = 3934, dst = 2227), (src = 5525, dst = 8056), (src = 3508, dst = 7186), (src = 7485, dst = 7684), (src = 45, dst = 5662), (src = 4980, dst = 5649), (src = 1636, dst = 2749), (src = 2083, dst = 26), (src = 1441, dst = 7737), (src = 1715, dst = 6973), (src = 6912, dst = 9982), (src = 3498, dst = 6984), (src = 4665, dst = 3853), (src = 9733, dst = 9480), (src = 4123, dst = 5364), (src = 552, dst = 3122), (src = 7298, dst = 4899), (src = 1854, dst = 6147), (src = 4017, dst = 6691), (src = 6349, dst = 2025), (src = 7933, dst = 1219), (src = 9975, dst = 3404), (src = 1951, dst = 7664), (src = 6562, dst = 8352), (src = 7193, dst = 8775), (src = 8596, dst = 1455), (src = 2257, dst = 5182), (src = 4136, dst = 2807), (src = 1049, dst = 7290), (src = 6411, dst = 6181), (src = 5406, dst = 5295), (src = 6395, dst = 3703

3754), (src = 4135, dst = 5129), (src = 9172, dst = 8875), (src = 8531, dst = 3432), (src = 6224, dst = 4743), (src = 3589, dst = 3909), (src = 7666, dst = 5359), (src = 3344, dst = 5474), (src = 6685, dst = 2048), (src = 6430, dst = 2408), (src = 9682, dst = 7910), (src = 3689, dst = 2214), (src = 8466, dst = 8577), (src = 6456, dst = 9581), (src = 7, dst = 7107), (src = 6335, dst = 6877), (src = 4188, dst = 188), (src = 4970, dst = 8753), (src = 5980, dst = 5944), (src = 530, dst = 9735), (src = 470, dst = 401), (src = 933, dst = 6704), (src = 8842, dst = 6190), (src = 8125, dst = 9755), (src = 7997, dst = 4780), (src = 9846, dst = 6193), (src = 4518, dst = 2166), (src = 8287, dst = 8925), (src = 1507, dst = 8854), (src = 7347, dst = 5305), (src = 8991, dst = 1508), (src = 4418, dst = 77), (src = 2401, dst = 8532), (src = 1587, dst = 4148), (src = 6757, dst = 7781), (src = 2357, dst = 5034), (src = 8897, dst = 3780), (src = 9865, dst = 9115), (src = 353, dst = 3782), (src = 8133, dst